In [1]:
import numpy as np
import sys
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf
import os
import time

from keras.models import Model
from keras.optimizers import SGD #optimizador del Gradiente Descendente (SGD)
from keras.layers import Input, Dense, SimpleRNN
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras import backend as K 

In [2]:
fileDL= tf.keras.utils.get_file('TrainText.txt','https://raw.githubusercontent.com/JoseSolano217/DeepLearningP1/main/Deep_Learning_P1/Text/TrainText.txt')
texto = open(fileDL, 'rb').read().decode(encoding='latin-1')
print(texto)

49152/43533 [=================================] - 0s 0us/step
Snow falls on the Hakurei Shrine in winter. It had been destroyed by an earthquake before, but now it's back to normal.
One day, something happened to surprise the Hakurei Shrine's shrine maiden, Reimu Hakurei. That was the sudden appearance of a white pillar shooting into the air - a geyser - near the shrine grounds. 
Normally, when something unusual happens, she goes to find out what happened and quickly bring the matter to a close, but this made her happy as much as it surprised her. After all, the geyser would melt the snow and become a hot spring that would ease the fatigue of both humans and youkai from nearby. 
Marisa: " ... isn't it dangerous to get too close to a geyser?" 
Reimu: "It could attract visitors to the shrine, so who cares? If a hot spring comes up then even villagers will come here, mark my words." 
Marisa: "Yeah, for the hot spring." 
However, Reimu's hopeful observations were off the mark.
The geyser s

In [3]:
print('el texto tiene longitud de {} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de {} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de 43533 caracteres
el texto esta compuesto de 80 caracteres
['\n', '\r', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x94', '\x99', '©', 'Ã', 'â', 'ã']


In [4]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'Snow falls on the Hakurei Shrine in winter. It had been destroyed by an earthquake before, but now i'
array([39, 60, 61, 69,  2, 52, 47, 58, 58, 65,  2, 61, 60,  2, 66, 54, 51,
        2, 29, 47, 57, 67, 64, 51, 55,  2, 39, 54, 64, 55, 60, 51,  2, 55,
       60,  2, 69, 55, 60, 66, 51, 64, 10,  2, 30, 66,  2, 54, 47, 50,  2,
       48, 51, 51, 60,  2, 50, 51, 65, 66, 64, 61, 71, 51, 50,  2, 48, 71,
        2, 47, 60,  2, 51, 47, 64, 66, 54, 63, 67, 47, 57, 51,  2, 48, 51,
       52, 61, 64, 51,  8,  2, 48, 67, 66,  2, 60, 61, 69,  2, 55])


In [5]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=100
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Snow falls on the Hakurei Shrine in winter. It had been destroyed by an earthquake before, but now it'
"'s back to normal.\r\nOne day, something happened to surprise the Hakurei Shrine's shrine maiden, Reimu"
' Hakurei. That was the sudden appearance of a white pillar shooting into the air - a geyser - near th'
'e shrine grounds. \r\nNormally, when something unusual happens, she goes to find out what happened and '
'quickly bring the matter to a close, but this made her happy as much as it surprised her. After all, '
'the geyser would melt the snow and become a hot spring that would ease the fatigue of both humans and'
' youkai from nearby. \r\nMarisa: " ... isn\'t it dangerous to get too close to a geyser?" \r\nReimu: "It c'
'ould attract visitors to the shrine, so who cares? If a hot spring comes up then even villagers will '
'come here, mark my words." \r\nMarisa: "Yeah, for the hot spring." \r\nHowever, Reimu\'s hopeful observati'
'ons were off the mark.\r\nThe geyser spewed forth

In [6]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'Snow falls on the Hakurei Shrine in winter. It had been destroyed by an earthquake before, but now i'
Target data:  'now falls on the Hakurei Shrine in winter. It had been destroyed by an earthquake before, but now it'


In [7]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [8]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=
#    -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20480     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 80)            82000     
Total params: 22,134,864
Trainable params: 22,134,864
Non-trainable params: 0
_________________________________________________________________


In [9]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Loss y el optimizador.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

In [10]:
checkpoint_dir='/content/checkpoints'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True, 
    save_freq=25
)

In [11]:
# Load Weights
model.load_weights("/content/drive/MyDrive/ModelRNN/model_backup/Copia de model.h5")

# Compile Model
model.compile(optimizer='adam',loss=loss)

#Training zone

In [ ]:
EPOCHS= 470
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/470
6/6 [==============================] - 11s 238ms/step - loss: 0.0359
Epoch 2/470
6/6 [==============================] - 2s 239ms/step - loss: 0.0346
Epoch 3/470
6/6 [==============================] - 1s 240ms/step - loss: 0.0345
Epoch 4/470
6/6 [==============================] - 1s 241ms/step - loss: 0.0352
Epoch 5/470
6/6 [==============================] - 2s 371ms/step - loss: 0.0362
Epoch 6/470
6/6 [==============================] - 1s 242ms/step - loss: 0.0353
Epoch 7/470
6/6 [==============================] - 1s 240ms/step - loss: 0.0354
Epoch 8/470
6/6 [==============================] - 1s 244ms/step - loss: 0.0361
Epoch 9/470
6/6 [==============================] - 2s 345ms/step - loss: 0.0363
Epoch 10/470
6/6 [==============================] - 1s 244ms/step - loss: 0.0367
Epoch 11/470
6/6 [==============================] - 2s 245ms/step - loss: 0.0369
Epoch 12/470
6/6 [==============================] - 1s 245ms/step - loss: 0.0373
Epoch 13/470
6/6 [==================

In [ ]:
#Generar nuevo texto
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
#model.load_weights("/content/drive/MyDrive/ModelRNN/model.h5")
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=3000
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 2.0  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"X then meets Utsuho"))

In [ ]:
char_json = 
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("/content/drive/MyDrive/ModelRNN/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/ModelRNN/model.hdf5")
model.save('/content/drive/MyDrive/ModelRNN/model.h5')
model.compile(optimizer='adam',loss=loss)
print("modelo salvado en disco")